In [ ]:
#######################################################################
##  Made by: Dr. Keungoui Kim
##  Title: Scitech data import from Dimensions
##  goal : 
##  Data set:  
##  Time Span:
##  Variables
##      Input: 
##      Output: 
##  Time-stamp: #  "Sun Jan 26 01:47:34 2020":  edited by awe kim ; code
##  Notice :
#######################################################################

In [1]:
### Library
import pandas as pd
import numpy as np
import math

### Directory Setting for Export
#dir = "E:/Google Drive (awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # Home
dir = "D:/Google Drive(awekim@handong.edu)/[Research]/00_Dimensions/Dimension_files/" # HGU

In [2]:
### API log in
import dimcli
dimcli.login(key="792DDFAFCCA7478D8F37159F274A2783",
             endpoint="https://app.dimensions.ai/api/dsl/v2")

dsl = dimcli.Dsl()

Dimcli - Dimensions API Client (v0.9.9.1)
Connected to: <https://app.dimensions.ai/api/dsl/v2> - DSL v2.5
Method: manual login


In [ ]:
# Extract Grant data
# Extract Funding data 
# What funding initiates science -> technology

In [5]:
### 
grant_data = dsl.query_iterative("""search grants
                                  return grants limit 1000""") #  limit 50000

Returned Grants: 1000 (total = 6381797)
Time: 4.54s


In [6]:
grant_data_df = grant_data.as_dataframe()
print(grant_data_df.shape)
print(grant_data_df.id.nunique())

(1000, 11)
1000


In [69]:
grant_data_df.head()

,id,title,active_year,end_date,funder_org_name,funder_orgs,grant_number,language,original_title,start_date,start_year
0,grant.9852804,"Multi-scale, multi-modal X-ray imaging using s...","[2023, 2024, 2025, 2026]",2026-12-30,Australian Research Council,"[{'acronym': 'ARC', 'city_name': 'Canberra', '...",DE220101402,en,"Multi-scale, multi-modal X-ray imaging using s...",2023-12-31,2023
1,grant.12981162,Defining the Pathogenic Role of Neural Crest C...,"[2023, 2024, 2025]",2025-11-30,TSC Alliance,"[{'acronym': 'TS Alliance', 'city_name': 'Silv...",990240,en,Defining the Pathogenic Role of Neural Crest C...,2023-12-01,2023
2,grant.12956342,Network slicing and fairness over optical tran...,"[2023, 2024]",2024-11-30,São Paulo Research Foundation,"[{'acronym': 'FAPESP', 'city_name': 'São Paulo...",19/26130-4,en,Network slicing and fairness over optical tran...,2023-12-01,2023
3,grant.12930187,3D printing multifunctional devices without in...,"[2023, 2024, 2025]",2025-05-30,Engineering and Physical Sciences Research Cou...,"[{'acronym': 'EPSRC', 'city_name': 'Swindon', ...",EP/W034409/1,en,3D printing multifunctional devices without in...,2023-12-01,2023
4,grant.12915816,Iron gall ink challenges. Methodologies for id...,"[2023, 2024]",2024-01-31,Fundação para a Ciência e Tecnologia,"[{'acronym': 'FCT', 'city_name': 'Lisbon', 'co...",COVID/BD/153467/2023,en,Iron gall ink challenges. Methodologies for id...,2023-12-01,2023


In [71]:
grant_data_df['funder_orgs'][0]

[{'acronym': 'ARC',
  'city_name': 'Canberra',
  'country_name': 'Australia',
  'id': 'grid.413452.5',
  'latitude': -35.282425,
  'linkout': ['http://www.arc.gov.au/'],
  'longitude': 149.12405,
  'name': 'Australian Research Council',
  'state_name': 'Australian Capital Territory',
  'types': ['Government']}]

In [62]:
query = """search patents 
        where associated_grant_ids is not empty
        return patents[id+associated_grant_ids+funders]  limit 100 """
temp = dsl.query(query) #  limit 50000

pat_df = temp.as_dataframe()

Returned Patents: 100 (total = 159008)
Time: 1.43s


In [59]:
temp_df

,id,associated_grant_ids,funders
0,ZA-201603129-B,[grant.7612229],"[{'acronym': 'MRC', 'city_name': 'London', 'co..."
1,WO-2021191474-A1,"[grant.2773993, grant.3957614]","[{'acronym': 'MRC', 'city_name': 'London', 'co..."
2,WO-2021185873-A1,[grant.7444838],"[{'acronym': 'MRC', 'city_name': 'London', 'co..."
3,WO-2021176236-A1,"[grant.7924629, grant.7748473, grant.9698640, ...","[{'acronym': 'BBSRC', 'city_name': 'Swindon', ..."
4,WO-2021176233-A1,[grant.9288125],"[{'acronym': 'MRC', 'city_name': 'London', 'co..."
...,...,...,...
95,WO-2020148548-A1,[grant.2784246],"[{'acronym': 'MRC', 'city_name': 'London', 'co..."
96,WO-2020148425-A1,[grant.3958177],"[{'acronym': 'MRC', 'city_name': 'London', 'co..."
97,WO-2020144467-A1,"[grant.7443727, grant.5498139]","[{'acronym': 'EPSRC', 'city_name': 'Swindon', ..."
98,WO-2020128496-A1,"[grant.2756358, grant.8470217, grant.7747241, ...","[{'acronym': 'BBSRC', 'city_name': 'Swindon', ..."


In [47]:
temp_df['funders'][0]

[{'acronym': 'MRC',
  'city_name': 'London',
  'country_name': 'United Kingdom',
  'id': 'grid.14105.31',
  'latitude': 51.514755,
  'linkout': ['https://mrc.ukri.org/'],
  'longitude': -0.119432,
  'name': 'Medical Research Council',
  'types': ['Government']}]

In [66]:
pat_funder_df = pd.DataFrame()
for i in range(pat_df.shape[0]):
    temp_data = pd.DataFrame.from_records(pat_df['funders'][i])
    temp_data['id'] = pat_df['id'][i]
    pat_funder_df = pd.concat([pat_funder_df, temp_data], axis=0)


In [64]:
pat_df.shape[0]

100

In [67]:
pat_funder_df

,acronym,city_name,country_name,id,latitude,linkout,longitude,name,types,state_name
0,MRC,London,United Kingdom,ZA-201603129-B,51.514755,[https://mrc.ukri.org/],-0.119432,Medical Research Council,[Government],NaN
0,MRC,London,United Kingdom,WO-2021191474-A1,51.514755,[https://mrc.ukri.org/],-0.119432,Medical Research Council,[Government],NaN
0,MRC,London,United Kingdom,WO-2021185873-A1,51.514755,[https://mrc.ukri.org/],-0.119432,Medical Research Council,[Government],NaN
0,BBSRC,Swindon,United Kingdom,WO-2021176236-A1,51.566742,[http://www.bbsrc.ac.uk/],-1.784771,Biotechnology and Biological Sciences Research...,[Government],Wiltshire
0,MRC,London,United Kingdom,WO-2021176233-A1,51.514755,[https://mrc.ukri.org/],-0.119432,Medical Research Council,[Government],NaN
...,...,...,...,...,...,...,...,...,...,...
0,MRC,London,United Kingdom,WO-2020148548-A1,51.514755,[https://mrc.ukri.org/],-0.119432,Medical Research Council,[Government],NaN
0,MRC,London,United Kingdom,WO-2020148425-A1,51.514755,[https://mrc.ukri.org/],-0.119432,Medical Research Council,[Government],NaN
0,EPSRC,Swindon,United Kingdom,WO-2020144467-A1,51.567093,[https://www.epsrc.ac.uk/],-1.784602,Engineering and Physical Sciences Research Cou...,[Government],England
0,BBSRC,Swindon,United Kingdom,WO-2020128496-A1,51.566742,[http://www.bbsrc.ac.uk/],-1.784771,Biotechnology and Biological Sciences Research...,[Government],Wiltshire


In [68]:
temp_data

,acronym,city_name,country_name,id,latitude,linkout,longitude,name,state_name,types
0,EPSRC,Swindon,United Kingdom,WO-2020126459-A1,51.567093,[https://www.epsrc.ac.uk/],-1.784602,Engineering and Physical Sciences Research Cou...,England,[Government]
